In [22]:
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import csv
import time
import os
import logging

In [23]:
# # connect to the sim 
# client = carla.Client('localhost', 2000)

try:
    # Connect to the server on localhost at port 2000
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)  # seconds

    # Retrieve the worl
    print("Connected to CARLA world:")

except Exception as e:
    print("Failed to connect to CARLA:", e)

Connected to CARLA world:


In [24]:
# optional to load different towns
#client.set_timeout(15)
client.load_world('Town01')

In [25]:
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [26]:
# Cell 1: Setup Traffic Manager
tm_port = 8000
traffic_manager = client.get_trafficmanager(tm_port)
traffic_manager.set_global_distance_to_leading_vehicle(2.5)

# Optional: enable hybrid mode for large maps
traffic_manager.set_hybrid_physics_mode(True)
traffic_manager.set_hybrid_physics_radius(70.0)
print("Traffic Manager configured.")

Traffic Manager configured.


In [27]:
# Cell 2: Configure synchronous mode
settings = world.get_settings()
synchronous_master = False

if not settings.synchronous_mode:
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.05
    synchronous_master = True
    world.apply_settings(settings)
print(f"Synchronous mode enabled: {settings.synchronous_mode}, fixed delta seconds: {settings.fixed_delta_seconds}")

Synchronous mode enabled: True, fixed delta seconds: 0.05


In [28]:
# Cell 3: Prepare spawn points and vehicle blueprints
import random

vehicle_blueprints = world.get_blueprint_library().filter('vehicle.*')

random.shuffle(spawn_points)

number_of_vehicles = min(30, len(spawn_points))
print(f"Number of vehicles to spawn: {number_of_vehicles}")


Number of vehicles to spawn: 30


In [29]:
# Cell 4: Spawn vehicles with autopilot
vehicles_list = []
batch = []

for n in range(number_of_vehicles):
    bp = random.choice(vehicle_blueprints)
    if bp.has_attribute('color'):
        color = random.choice(bp.get_attribute('color').recommended_values)
        bp.set_attribute('color', color)
    if bp.has_attribute('driver_id'):
        driver_id = random.choice(bp.get_attribute('driver_id').recommended_values)
        bp.set_attribute('driver_id', driver_id)
    bp.set_attribute('role_name', 'autopilot')
    transform = spawn_points[n]
    if n == 0:
        # For the first vehicle: only spawn (no autopilot yet)
        batch.append(carla.command.SpawnActor(bp, transform))
    else:
        # For others: spawn and set autopilot immediately
        batch.append(carla.command.SpawnActor(bp, transform).then(
            carla.command.SetAutopilot(carla.command.FutureActor, True, tm_port)
        ))
        
responses = client.apply_batch_sync(batch, synchronous_master)
for response in responses:
    if response.error:
        import logging
        logging.error(response.error)
    else:
        vehicles_list.append(response.actor_id)

print(f"Spawned {len(vehicles_list)} vehicles.")


Spawned 30 vehicles.


In [30]:
# Cell 5: Prepare to spawn walkers
walker_blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')
number_of_walkers = 10
walkers_list = []
spawn_points = []

for _ in range(number_of_walkers):
    loc = world.get_random_location_from_navigation()
    if loc is not None:
        spawn_points.append(carla.Transform(loc))

print(f"Walker spawn points generated: {len(spawn_points)}")

Walker spawn points generated: 10


In [31]:
# Cell 6: Spawn walkers (pedestrians)
walker_speed = []
batch = []

for spawn_point in spawn_points:
    walker_bp = random.choice(walker_blueprints)
    if walker_bp.has_attribute('is_invincible'):
        walker_bp.set_attribute('is_invincible', 'false')
    speed = walker_bp.get_attribute('speed').recommended_values[1]
    walker_speed.append(float(speed))
    batch.append(carla.command.SpawnActor(walker_bp, spawn_point))

results = client.apply_batch_sync(batch, True)
for i, result in enumerate(results):
    if result.error:
        logging.error(result.error)
    else:
        walkers_list.append({"id": result.actor_id})
print(f"Spawned {len(walkers_list)} walkers.")


ERROR:root:Spawn failed because of collision at spawn position
ERROR:root:Spawn failed because of collision at spawn position


Spawned 8 walkers.


In [32]:
# Cell 7: Spawn controllers for walkers
controller_bp = world.get_blueprint_library().find('controller.ai.walker')
batch = []

for walker in walkers_list:
    batch.append(carla.command.SpawnActor(controller_bp, carla.Transform(), walker["id"]))

results = client.apply_batch_sync(batch, True)
for i, result in enumerate(results):
    if result.error:
        logging.error(result.error)
    else:
        walkers_list[i]["con"] = result.actor_id

print("Spawned controllers for walkers.")


Spawned controllers for walkers.


In [33]:
# Cell 8: Start walkers and assign random destinations
all_id = []
for w in walkers_list:
    all_id.extend([w["con"], w["id"]])
all_actors = world.get_actors(all_id)

world.tick()

for i in range(0, len(all_id), 2):
    controller = all_actors[i]
    actor = all_actors[i + 1]
    controller.start()
    controller.go_to_location(world.get_random_location_from_navigation())
    controller.set_max_speed(walker_speed[i // 2])

print(f"Started {len(walkers_list)} walkers with destinations.")


Started 8 walkers with destinations.


In [34]:
# Cell 2.1: Keep ticking the world in synchronous mode
import time

print("Starting ticking loop. Press Interrupt (Ctrl+C) to stop.")
try:
    while True:
        world.tick()
        time.sleep(0.05)  # match fixed_delta_seconds to avoid busy wait
except KeyboardInterrupt:
    print("Ticking loop stopped by user.")

Starting ticking loop. Press Interrupt (Ctrl+C) to stop.
Ticking loop stopped by user.


In [35]:
# move simulator view to the car
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[2]
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [36]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*Audi*')

In [37]:
#spawn a car in a random location (first spawn point in the list)
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [38]:
# Setup bird eye view

CAMERA_POS_Z = 15       # 15 meters above ground
CAMERA_ROT_X = -90.0    # Camera looks straight down

# === Get blueprint library ===
blueprint_library = world.get_blueprint_library()

# === RGB Camera blueprint ===
camera_rgb_bp = blueprint_library.find('sensor.camera.rgb')
camera_rgb_bp.set_attribute('image_size_x', '640')
camera_rgb_bp.set_attribute('image_size_y', '360')

# === Camera Transform ===
camera_init_trans = carla.Transform(
    carla.Location(z=CAMERA_POS_Z),
    carla.Rotation(pitch=CAMERA_ROT_X)
)

# === Spawn RGB Camera actor ===
camera_rgb = world.spawn_actor(camera_rgb_bp, camera_init_trans, attach_to=vehicle)

# === Semantic Segmentation Camera blueprint ===
camera_seg_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
camera_seg_bp.set_attribute('image_size_x', '640')
camera_seg_bp.set_attribute('image_size_y', '360')

# === Spawn Semantic Segmentation Camera actor ===
camera_seg = world.spawn_actor(camera_seg_bp, camera_init_trans, attach_to=vehicle)

# === Lidar Camera blueprint ===
camera_lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
camera_lidar_bp.set_attribute('channels', '32')
camera_lidar_bp.set_attribute('range', '50')
camera_lidar_bp.set_attribute('rotation_frequency', '10')

# === Spawn Lidar Camera actor ===
camera_lidar = world.spawn_actor(camera_lidar_bp, camera_init_trans, attach_to=vehicle)

# === Depth Camera blueprint ===
camera_depth_bp = blueprint_library.find('sensor.camera.depth')
camera_depth_bp.set_attribute('image_size_x', '640')
camera_depth_bp.set_attribute('image_size_y', '360')

# === Spawn Depth Camera actor ===
camera_depth = world.spawn_actor(camera_depth_bp, camera_init_trans, attach_to=vehicle)


In [39]:
# Setup window car view

CAMERA_POS_Z = 1       # 15 meters above ground

# === Camera Transform ===
camera_init_trans_window = carla.Transform(
    carla.Location(z=CAMERA_POS_Z)
)

# === Spawn RGB Camera actor ===
camera_rgb_window = world.spawn_actor(camera_rgb_bp, camera_init_trans_window, attach_to=vehicle)

# === Spawn Semantic Segmentation Camera actor ===
camera_seg_window = world.spawn_actor(camera_seg_bp, camera_init_trans_window, attach_to=vehicle)

# === Spawn Lidar Camera actor ===
camera_lidar_window = world.spawn_actor(camera_lidar_bp, camera_init_trans_window, attach_to=vehicle)

# === Spawn Depth Camera actor ===
camera_depth_window = world.spawn_actor(camera_depth_bp, camera_init_trans_window, attach_to=vehicle)


In [40]:
os.makedirs('D:/KP_Alexis_Doci/lidar_frames', exist_ok=True)

In [41]:
# bird eye view
os.makedirs('D:/KP_Alexis_Doci/rgb', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/seg', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/rgb', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/seg', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/lidar', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/depth', exist_ok=True)

# window car view
os.makedirs('D:/KP_Alexis_Doci/rgb_window', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/seg_window', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/lidar_window', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/depth_window', exist_ok=True)
os.makedirs('D:/KP_Alexis_Doci/lidar_frames', exist_ok=True)



rgb_frame_counter = 1
seg_frame_counter = 1
depth_frame_counter = 1
lidar_frame_counter = 1

rgb_frame_counter_window = 1
seg_frame_counter_window = 1
depth_frame_counter_window = 1
lidar_frame_counter_window = 1

# === Bird Eye view callbacks ===
def process_rgb(image):
    global rgb_frame_counter
    image.save_to_disk(f'D:/KP_Alexis_Doci/rgb/frame_{rgb_frame_counter:05d}.png')
    rgb_frame_counter += 1

def process_seg(image):
    global seg_frame_counter
    image.convert(carla.ColorConverter.CityScapesPalette)
    image.save_to_disk(f'D:/KP_Alexis_Doci/seg/frame_{seg_frame_counter:05d}.png')
    seg_frame_counter += 1

def process_depth(image):
    global depth_frame_counter
    image.convert(carla.ColorConverter.LogarithmicDepth)
    image.save_to_disk(f'D:/KP_Alexis_Doci/depth/frame_{depth_frame_counter:05d}.png')
    depth_frame_counter += 1


# === Window Car view callbacks ===
def process_rgb_window(image):
    global rgb_frame_counter_window
    image.save_to_disk(f'D:/KP_Alexis_Doci/rgb_window/frame_{rgb_frame_counter_window:05d}.png')
    rgb_frame_counter_window += 1

def process_seg_window(image):
    global seg_frame_counter_window
    image.convert(carla.ColorConverter.CityScapesPalette)
    image.save_to_disk(f'D:/KP_Alexis_Doci/seg_window/frame_{seg_frame_counter_window:05d}.png')
    seg_frame_counter_window += 1

def process_depth_window(image):
    global depth_frame_counter_window
    image.convert(carla.ColorConverter.LogarithmicDepth)
    image.save_to_disk(f'D:/KP_Alexis_Doci/depth_window/frame_{depth_frame_counter_window:05d}.png')
    depth_frame_counter_window += 1

In [44]:
!pip install matplotlib
!pip install open3d numpy matplotlib


  Using cached matplotlib-3.5.3-cp37-cp37m-win_amd64.whl (7.2 MB)
  Using cached pyparsing-3.1.4-py3-none-any.whl (104 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached Pillow-9.5.0-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached kiwisolver-1.4.5-cp37-cp37m-win_amd64.whl (55 kB)
  Using cached open3d-0.17.0-cp37-cp37m-win_amd64.whl (62.2 MB)
  Using cached Werkzeug-2.2.3-py3-none-any.whl (233 kB)
  Using cached configargparse-1.7.1-py3-none-any.whl (25 kB)
  Using cached nbformat-5.7.0-py3-none-any.whl (77 kB)
  Using cached dash-2.15.0-py3-none-any.whl (10.2 MB)
  Using cached Flask-2.2.5-py3-none-any.whl (101 kB)
  Using cached plotly-5.18.0-py3-none-any.whl (15.6 MB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached itsdangerous-2.1.2-py3-none-a

In [45]:
import matplotlib.pyplot as plt
import open3d as o3d

lidar_frame_counter = 0

def process_lidar(point_cloud):
    import os
    import open3d as o3d
    import numpy as np
    import matplotlib.pyplot as plt

    global lidar_frame_counter

    os.makedirs('D:/KP_Alexis_Doci/lidar_frames', exist_ok=True)
    print(f"📥 Processing lidar frame {lidar_frame_counter}...")

    data = np.frombuffer(point_cloud.raw_data, dtype=np.float32).reshape(-1, 4)
    xyz = data[:, :3]
    intensities = data[:, 3]

    # Normalisasi intensitas (untuk warna)
    norm_intensity = (intensities - intensities.min()) / (intensities.ptp() + 1e-6)
    colors = plt.get_cmap('plasma')(norm_intensity)[:, :3]

    # Buat point cloud open3d
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Simpan gambar dari point cloud (sebagai PNG)
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    vis.add_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    save_path = f'D:/KP_Alexis_Doci/lidar_frames/lidar_{lidar_frame_counter:05d}.png'
    vis.capture_screen_image(save_path)
    vis.destroy_window()

    print(f"[✓] Lidar frame saved: {save_path}")
    lidar_frame_counter += 1


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [48]:
# pemrosesan khusus lidar
import math
import open3d as o3d
from matplotlib import cm

# ==== Color Map dan Fungsi Axis Open3D ====
# Colormap untuk Lidar
VIRIDIS = cm.get_cmap('plasma').resample(256)(np.linspace(0, 1, 256))[:, :3]
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

# Colormap untuk Radar
COOL_RANGE = np.linspace(0.0, 1.0, COOL.shape[0])
COOL = cm.get_cmap('winter').resample(256)(np.linspace(0, 1, 256))[:, :3]


# Fungsi untuk menambahkan axis di Open3D
def add_open3d_axis(vis):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0, 0, 0], [1, 0, 0],
        [0, 1, 0], [0, 0, 1]
    ]))
    axis.lines = o3d.utility.Vector2iVector([[0,1],[0,2],[0,3]])
    axis.colors = o3d.utility.Vector3dVector([[1,0,0], [0,1,0], [0,0,1]])
    vis.add_geometry(axis)

# === Callback untuk Sensor ===
# Lidar
def lidar_callback(point_cloud, point_list):
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.float32)).reshape(-1, 4)
    intensity = data[:, -1]
    int_color = np.c_[
        np.interp(intensity, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity, VID_RANGE, VIRIDIS[:, 2])
    ]
    points = data[:, :-1]
    points[:, :1] = -points[:, :1]  # flip x
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

# Radar
def radar_callback(data, radar_list):
    radar_data = np.zeros((len(data), 4))
    for i, d in enumerate(data):
        x = d.depth * math.cos(d.altitude) * math.cos(d.azimuth)
        y = d.depth * math.cos(d.altitude) * math.sin(d.azimuth)
        z = d.depth * math.sin(d.altitude)
        radar_data[i, :] = [x, y, z, d.velocity]
    intensity = np.abs(radar_data[:, -1])
    int_color = np.c_[
        np.interp(intensity, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity, COOL_RANGE, COOL[:, 2])
    ]
    points = radar_data[:, :-1]
    points[:, :1] = -points[:, :1]
    radar_list.points = o3d.utility.Vector3dVector(points)
    radar_list.colors = o3d.utility.Vector3dVector(int_color)

# Kamera RGB
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(
        np.copy(image.raw_data), (image.height, image.width, 4))



[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01523.png
📥 Processing lidar frame 1524...


AttributeError: 'ListedColormap' object has no attribute 'resample'

[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01524.png
📥 Processing lidar frame 1525...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01525.png
📥 Processing lidar frame 1526...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01526.png
📥 Processing lidar frame 1527...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01527.png
📥 Processing lidar frame 1528...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01528.png
📥 Processing lidar frame 1529...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01529.png
📥 Processing lidar frame 1530...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01530.png
📥 Processing lidar frame 1531...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01531.png
📥 Processing lidar frame 1532...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01532.png
📥 Processing lidar frame 1533...
[✓] Lidar frame saved: D:/KP_Alexis_Doci/lidar_frames/lidar_01533.png
📥 P

In [46]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

In [ ]:
import csv
import time

# Prepare a CSV file for logging
log_filename = f'car_location_log_{time.strftime("%Y%m%d-%H%M%S")}.csv'
log_file = open(log_filename, 'w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['frame', 'timestamp', 'x', 'y', 'z'])

# Start sensor listeners
# --- Bird's-eye view ---
camera_rgb.listen(process_rgb)
camera_seg.listen(process_seg)
camera_depth.listen(process_depth)
camera_lidar.listen(process_lidar)

# --- Window view ---
camera_rgb_window.listen(process_rgb_window)
camera_seg_window.listen(process_seg_window)
camera_depth_window.listen(process_depth_window)
camera_lidar_window.listen(process_lidar_window)

client.start_recorder("D:/KP_Alexis_Doci/recording01.log") 

# Define the asynchronous logging callback
def log_vehicle_location(snapshot):
    if vehicle is not None and vehicle.is_alive:
        vehicle_snapshot = snapshot.find(vehicle.id)
        if vehicle_snapshot:
            loc = vehicle_snapshot.get_transform().location
            timestamp = snapshot.timestamp

            # Write the data to the CSV file
            log_writer.writerow([
                snapshot.frame,
                timestamp.elapsed_seconds,
                loc.x,
                loc.y,
                loc.z
            ])

# Let the simulation run until user interruption
print("Logging vehicle location... Press Ctrl+C to stop.")
try:
    while True:
        world.tick()
        time.sleep(0.5)  # Keeps the script alive without blocking anything
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    # Stop all sensors
    camera_rgb.stop()
    camera_seg.stop()
    camera_depth.stop()
    camera_lidar.stop()
    camera_rgb_window.stop()
    camera_seg_window.stop()
    camera_depth_window.stop()
    camera_lidar_window.stop()

    client.stop_recorder()

    # Close the log file
    log_file.close()

    print("All sensors stopped and log file closed.")


In [ ]:
# client.replay_file("D:/KP_Alexis_Doci/recording01.log", 0, 0, 0)